In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from itertools import combinations
from tqdm.auto import tqdm
from collections import deque
from operator import itemgetter

In [2]:
rng = np.random.Generator(np.random.PCG64(42)) # random number generator for reproducibility

In [3]:
COUNTRY_LIST = ['Vanuatu', 'Italy', 'Russia', 'United States', 'China']
vanuatu = pd.read_csv('cities/vanuatu.csv', header=None, names=['city', 'latitude', 'longitude'])
italy = pd.read_csv('cities/italy.csv', header=None, names=['city', 'latitude', 'longitude'])
russia = pd.read_csv('cities/russia.csv', header=None, names=['city', 'latitude', 'longitude'])
us = pd.read_csv('cities/us.csv', header=None, names=['city', 'latitude', 'longitude'])
china = pd.read_csv('cities/china.csv', header=None, names=['city', 'latitude', 'longitude'])

In [4]:
def compute_distance(city1, city2):
    return geodesic(
        (city1.latitude, city1.longitude), (city2.latitude, city2.longitude)
        ).km

## Greedy algorithm

In [5]:
def create_dist_matrix(cities):
    dist_matrix = np.zeros((cities.shape[0], cities.shape[0]))
    for c1, c2 in combinations(cities.itertuples(), 2):
        dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = compute_distance(c1,c2)
    return dist_matrix

def greedy_tsp(cities):
    num_cities = cities.shape[0]
    dist_matrix = create_dist_matrix(cities)
    route = []
    current_city_index = 0
    route.append(current_city_index)
    cost = 0

    while not len(route)==num_cities:
        dist_matrix[:, current_city_index] = np.inf
        closest_city_index = np.argmin(dist_matrix[current_city_index])
        route.append(closest_city_index)
        cost += dist_matrix[current_city_index, closest_city_index]
        current_city_index = closest_city_index
        
    cost += compute_distance(cities.iloc[route[-1]], cities.iloc[route[0]])
    route.append(route[0])    
    return route, cost

## Greedier algorithm

In [6]:
def greedier_tsp(cities):

    min_cost = np.inf
    min_route = None
    num_cities = cities.shape[0]
    max_iter = np.min((num_cities, 15))

    # run the greedy algorithm with randomly selected starting city multiple times and keep the best solution
    for _ in tqdm(range(max_iter)):
        dist_matrix = create_dist_matrix(cities)
        route = []
        current_city_index = rng.integers(num_cities)
        route.append(current_city_index)
        cost = 0

        while not len(route)==num_cities:
            dist_matrix[:, current_city_index] = np.inf
            closest_city_index = np.argmin(dist_matrix[current_city_index])
            route.append(closest_city_index)
            cost += dist_matrix[current_city_index, closest_city_index]
            current_city_index = closest_city_index
        
        cost += compute_distance(cities.iloc[route[-1]], cities.iloc[route[0]])
        route.append(route[0])

        if cost < min_cost:
            min_cost = cost
            min_route = route


    return min_route, min_cost

## Tabu Search

In [7]:
def fitness(solution, dist_matrix):
    total_distance = 0
    for i in range(len(solution) - 1):
        city1 = solution[i]
        city2 = solution[i + 1]
        total_distance += dist_matrix[city1][city2]
        
    return total_distance

In [8]:
def inversion_mutation(solution):
    pos1 = rng.integers(0, len(solution)-2)
    pos2 = rng.integers(pos1, len(solution)-2)
    
    new_solution = solution.copy()
    new_solution.pop()
    new_solution[pos1:pos2+1] = new_solution[pos1:pos2+1][::-1]
    new_solution.append(new_solution[0])
    return new_solution

In [9]:
def scramble_mutation(solution):
    pos1 = rng.integers(0, len(solution)-2)
    pos2 = rng.integers(pos1, len(solution)-2)
    
    new_solution = solution.copy()
    new_solution.pop()
    subsequence = new_solution[pos1:pos2+1]
    rng.shuffle(subsequence)
    new_solution[pos1:pos2+1] = subsequence
    new_solution.append(new_solution[0])
    return new_solution

In [10]:
def generate_neighbor(solution):
    if rng.random() < 0.8:
        return inversion_mutation(solution)
    return scramble_mutation(solution)

In [11]:
def tabu_search(init_solution, dist_matrix, tabu_memory=50, max_worsening_moves=10, limit_iterations=10000, convergence_limit=500):

    current_solution = init_solution.copy()
    best_solution = init_solution.copy()
    current_fitness = fitness(current_solution, dist_matrix)
    best_fitness = current_fitness
    
    tabu_list = deque(maxlen=tabu_memory)
    tabu_list.append(tuple(current_solution))
    
    worsening_moves = 0
    iteration = 0
    no_improvement_counter = 0
    
    while no_improvement_counter < convergence_limit and iteration < limit_iterations:
        neighbor = generate_neighbor(current_solution)
        neighbor_fitness = fitness(neighbor, dist_matrix)
        
        if neighbor_fitness < best_fitness:
            best_solution = neighbor.copy()
            best_fitness = neighbor_fitness
            worsening_moves = 0
            no_improvement_counter = 0
        else:
            worsening_moves += 1
            no_improvement_counter += 1
        
        # accept neighbor if not tabu or if it's better than best known
        if tuple(neighbor) not in tabu_list or neighbor_fitness < best_fitness:
            current_solution = neighbor
            current_fitness = neighbor_fitness
            tabu_list.append(tuple(neighbor))
        
        # if too many worsening moves, return to best solution
        if worsening_moves >= max_worsening_moves:
            current_solution = best_solution.copy()
            current_fitness = best_fitness
            worsening_moves = 0
        
        iteration += 1
    
    return best_solution, best_fitness

## ES (Evolved Salesman)

In [12]:
def create_initial_population(init_solution, pop_size):
    population = [init_solution.copy()]
    
    while len(population) < pop_size:
        if rng.random() < 0.5:
            new_solution = inversion_mutation(init_solution)
        else:
            new_solution = scramble_mutation(init_solution)
        population.append(new_solution)
    
    return population

In [13]:
def rank_based_selection(population, fitness_values, num_parents):
    # lower fitness -> higher rank
    population_fitness = list(zip(population, fitness_values))
    population_fitness.sort(key=itemgetter(1))
    
    n = len(population_fitness)
    ranks = range(n, 0, -1)
    rank_sum = sum(ranks)
    probabilities = [r/rank_sum for r in ranks]
    
    selected_indices = rng.choice(n, size=num_parents, p=probabilities, replace=False)
    
    return [population_fitness[i][0] for i in selected_indices]

In [14]:
def inver_over_crossover(parent1, parent2):
    p1 = parent1[:-1].copy()
    p2 = parent2[:-1].copy()
    offspring = p1.copy()
    
    position1 = rng.integers(0, len(p1)-1)
    current_element = p1[position1]
    next_element = p1[(position1+1) % len(p1)]

    p2_current_pos = p2.index(current_element)
    p2_next_element = p2[(p2_current_pos + 1) % len(p2)]

    if p2_next_element == next_element:
        p2_next_element = p2[(p2_current_pos + 2) % len(p2)]
    
    position2 = p1.index(p2_next_element)
    
    #determine the inversion section
    if position1 < position2:
        offspring[position1+1:position2+1] = offspring[position1+1:position2+1][::-1]
    else:
        temp = offspring[position1+1:] + offspring[:position2+1]
        temp = temp[::-1]
        offspring[position1+1:] = temp[:len(offspring)-position1-1]
        offspring[:position2+1] = temp[len(offspring)-position1-1:]
    
    offspring.append(offspring[0])  
    return offspring

In [15]:
def evolution_strategy(init_solution, dist_matrix, pop_size=50, offspring_size=10, 
                      convergence_limit=1000, limit_num_generations=100000, mutation_prob=0.2):
    population = create_initial_population(init_solution, pop_size)
    fitness_values = [fitness(sol, dist_matrix) for sol in population]
    best_solution = min(zip(population, fitness_values), key=itemgetter(1))[0]
    best_fitness = min(fitness_values)
    fitness_history = [best_fitness]

    no_improvement_counter = 0
    generation = 0

    while no_improvement_counter < convergence_limit and generation < limit_num_generations:
        for _ in range(offspring_size):
            parents = rank_based_selection(population, fitness_values, 2)
            offspring = inver_over_crossover(parents[0], parents[1])

            if rng.random() < mutation_prob:
                if rng.random() < 0.7:
                    offspring = inversion_mutation(offspring)
                else:
                    offspring = scramble_mutation(offspring)
            
            offspring_fitness = fitness(offspring, dist_matrix)
            
            # Compare offspring with the worst solution in the population
            worst_index = max(range(len(fitness_values)), key=lambda i: fitness_values[i])
            if offspring_fitness < fitness_values[worst_index]:
                population[worst_index] = offspring
                fitness_values[worst_index] = offspring_fitness
                
                if offspring_fitness < best_fitness:
                    best_fitness = offspring_fitness
                    best_solution = offspring.copy()
                    no_improvement_counter = 0
                else:
                    no_improvement_counter += 1
            else:
                no_improvement_counter += 1
        
        fitness_history.append(best_fitness)
        generation += 1
    
    return best_solution, best_fitness, fitness_history

## Experimental results

In [16]:
for i, cities in enumerate([vanuatu, italy, russia, us, china]):
    dist_matrix = create_dist_matrix(cities)
    print(f"===== Country: {COUNTRY_LIST[i]} =====")

    greedy_solution, greedy_cost = greedy_tsp(cities)
    print(f"Greedy TSP distance: {greedy_cost}")
    greedier_solution, greedier_cost = greedier_tsp(cities)
    print(f"Greedier TSP distance: {greedier_cost}")
    tabu_solution, tabu_cost = tabu_search(greedy_solution, dist_matrix)
    print(f"Tabu search distance: {tabu_cost}")
    es_solution, es_cost, _ = evolution_strategy(tabu_solution, dist_matrix)
    print(f"ES distance: {es_cost}\n")

===== Country: Vanuatu =====
Greedy TSP distance: 1475.528091104531


  0%|          | 0/8 [00:00<?, ?it/s]

Greedier TSP distance: 1475.528091104531
Tabu search distance: 1345.5449564733115
ES distance: 1345.544956473311

===== Country: Italy =====
Greedy TSP distance: 4436.03176952516


  0%|          | 0/15 [00:00<?, ?it/s]

Greedier TSP distance: 4576.195155974603
Tabu search distance: 4429.095375854959
ES distance: 4429.095375854956

===== Country: Russia =====
Greedy TSP distance: 42334.16465744784


...

Greedier TSP distance: 40051.58704748039
Tabu search distance: 42334.16465744784
ES distance: 40802.40480925422

===== Country: United States =====
Greedy TSP distance: 48050.02586446137


  0%|          | 0/15 [00:00<?, ?it/s]

Greedier TSP distance: 46997.43371671413
Tabu search distance: 48050.02586446137
ES distance: 47284.889432599186

===== Country: China =====
Greedy TSP distance: 63962.9184294552


  0%|          | 0/15 [00:00<?, ?it/s]

Greedier TSP distance: 62439.5324998315
Tabu search distance: 63962.9184294552
ES distance: 63433.307812908984

